# Clasificador de texto con modelos *transformers*
Implementemos un clasificador usando un modelo BERT haciendo *fine-tuning* sobre un conjunto de análisis de sentimiento en Twitter.  

Usamos la librería `transformers` en su implementación para `Tensorflow`

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = None
import numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

from sklearn.model_selection import train_test_split

In [ ]:
#modelo a utilizar
nombre_modelo = 'bert-base-multilingual-uncased'

In [ ]:
# Leemos los datos
df = pd.read_csv('tweets_max.csv')

#seleccionamos columnas de interés
df = df[['content', 'polarity']]

#dejamos polaridades definidas
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

df.head()

In [ ]:
df.info()

## Limpieza de texto
Usamos Spacy para separar el texto en tokens y mantener sólo las palabras importantes, dejando su lemma

In [ ]:
import re, string

pattern1 = re.compile(r'@[\w_]+') #elimina menciones
pattern2 = re.compile(r'https?://[\w_./]+') #elimina URL
#pattern3 = re.compile(r'#[\w_]+') #elimina hashtags
pattern4 = re.compile('[{}]+'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text):
    """Limpiamos las menciones, URL y hashtags del texto. Luego 
    quitamos signos de puntuación"""
    text = pattern1.sub('mención', text)
    text = pattern2.sub('URL', text)
    #text = pattern3.sub('hashtag', text)
    text = pattern4.sub(' ', text)
    
    return text

In [ ]:
df['content'].iloc[0]

In [ ]:
clean_text(df['content'].iloc[0])

## Preparemos el conjunto de datos
Fijamos un tamaño máximo de vocabulario.  
Separamos los tweets en tokens dentro de este vocabulario y creamos las secuencias de longitud fija.  
La longitud de la secuencia viene dada por la longitud en tokens del tweet más largo. Sólo se conservan los tokens de las palabras en el vocabulario.

In [ ]:
#limpiamos texto y quitamos tweets que se han quedado vacíos
df.content=df.content.apply(clean_text)
df = df[df['content']!='']
#el conjunto de salida es la polaridad, hay que convertir a binario
#codificamos 'P' como 1 y 'N' se queda como 0
Y=(df.polarity=='P').values*1

## Creación del dataset
Convertimos los datos en un objeto dataset para entrenar los modelos. Primero tokenizamos al máximo tamaño de los documentos y convertimos a `dataset`.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)

encoded_input = tokenizer(df["content"].to_list(), padding=True, truncation=True, return_tensors="pt")

In [ ]:
encoded_input["input_ids"].shape

In [ ]:
from datasets import Dataset

ds = Dataset.from_dict({**encoded_input, "labels": Y})

In [ ]:
tokenized_datasets = ds.train_test_split(test_size=0.3)

In [ ]:
tokenized_datasets

## Entrenamos con la clase `PyTorch Trainer`
Es necesario tener instalada la librería `accelerate` de HuggingFace:  
```
conda install -c conda-forge accelerate
```

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(nombre_modelo, num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Tenemos que definir una métrica sobre la que la clase `Trainer` va a evaluar el modelo. Usamos la librería `evaluate` de HuggingFace para eso:  
```
conda install -c conda-forge evaluate
```

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Ahora definimos en una función cómo usar esta métrica para evaluar el modelo

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Podemos especificar que en el entrenamiento se reporte el resultado de cada época con estos parámetros:

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Entrenamiento
Creamos el objeto `Trainer` con el modelo, argumentos de entrenamiento, datasets de train y test y función de evaluación:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

Entrenamos el modelo

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(tokenized_datasets["test"]["labels"], preds, target_names=['N','P']))

In [ ]:
from scipy.special import softmax

pred_proba = softmax(predictions.predictions, 1)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(tokenized_datasets["test"]["labels"], pred_proba[:,1])

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

RocCurveDisplay.from_predictions(
    tokenized_datasets["test"]["labels"],
    pred_proba[:,1],
    name="Positive class",
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AUC curve")
plt.legend()
plt.show()

El modelo ya entrenado está en el atributo `model`:

In [ ]:
trainer.model

Podemos guardarlo para cargarlo más tarde, y usarlo para hacer inferencia con él

In [ ]:
id2label = {0: "Neg", 1: "Pos"}
trainer.model.config.id2label = id2label
trainer.model.config.label2id = {val: key for key, val in id2label.items()}

In [ ]:
trainer.model.save_pretrained('twitter_sentiment')

Podemos cargarlos más adelante

In [ ]:
modelo = AutoModelForSequenceClassification.from_pretrained('twitter_sentiment')

In [ ]:
modelo

In [ ]:
del modelo

O incluso podemos cargarlo como un pipeline para hacer inferencia fácilmente:

In [ ]:
from transformers import pipeline
pipe = pipeline('sentiment-analysis', model = 'twitter_sentiment', tokenizer=AutoTokenizer.from_pretrained(nombre_modelo))

In [ ]:
pipe("me quiero morir")